In [ ]:
import numpy as np


# Define Rastrigin Function


In [ ]:
def rastrigin(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

In [ ]:
input_values_2d = [
    [0, 0],
    [1, 1],
    [-1, -1],
    [2, 2],
    [-2, -2],
    [0.5, 0.5],
    [-0.5, -0.5],
    [3, 3],
    [-3, -3],
    [2.5, -2.5]
]
input_values_1d = [0, 1, -1, 2, -2, 0.5, -0.5, 3, -3, 2.5]


print(rastrigin(input_values_1d))
rastrigin(input_values_2d)

# Initiate population

In [9]:
def generate_particle(num_variables,swarm_size,x_min,x_max):
    X = []
    for i in range(swarm_size):
        Xi = []
        for j in range(num_variables):
            Xi.append(int(np.random.uniform(x_min,x_max)))  #testing with integers so easier to crosscheck
        X.append(np.array(Xi))

    velocity =  np.array(X)
    velocity[:] = 0
    
    return np.array(X),np.array(velocity)

In [15]:
particle_test,velocity_test = generate_particle(2,10,-5,5)
# print(particle_test)
# print(velocity_test)

# rastrigin(particle_test)
for i in particle_test:
    print(rastrigin(i))
my_dict = {tuple(i) : rastrigin(i) for i in particle_test}
print(my_dict)

TypeError: object of type 'numpy.int32' has no len()

In [ ]:
def local_best(X):
    
    pass

# Particle Swarm Optimisation

1: swarmsize ← desired swarm size 

2: α ← proportion of velocity to be retained

3: β ← proportion of personal best to be retained

4: γ ← proportion of the informants’ best to be retained

5: δ ← proportion of global best to be retained

6: e ← jump size of a particle


In [ ]:
def pso(num_iterations,swarm_size,num_variables,x_min,x_max,alpha,beta,gamma,delta,epsilon):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)
    
    loc_best = []
    glo_best = 0
    
    for i in range(num_iterations):
        fitness = rastrigin(swarm)
        if i == 1:
            loc_best = swarm
            
        else:
            for j in range(swarm_size):
                if rastrigin(swarm[j]) < fitness[j]:
                    loc_best[j] = swarm[j]

                
                
                
                    
    
    pass   